In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Sjögrens_Syndrome/GSE135809'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome data of B cells in patients with primary Sjögren's syndrome"
!Series_summary	"We compared subsets of B cells as follows: Bm1 cells; CD38-IgD+, naïve B cells; CD38+IgD+, pre-germinal centre B cells; CD38highIgD+ and memory B cells; CD38±IgD that were collected from patients with primary Sjögren's syndrome."
!Series_summary	"As a result, list of 623 differentially expressed genes was created. We found  interferon signature genes and HLA genes were mostly up-regulated in patients compared to healthy controls."
!Series_overall_design	"Examination between patients with primary Sjögren's syndrome and healthy controls"
Sample Characteristics Dictionary:
{0: ['subject status: healthy control'], 1: ['subject id: HC-1', 'subject id: HC-2', 'subject id: HC-3', 'subject id: HC-4', 'subject id: HC-5', 'subject id: HC-6', 'subject id: pSS-1', 'subject id: pSS-2', 'subject id: pSS-3', 'subject id: pSS-4', 'subject id: pSS-5', 'subject id: pSS-6'], 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Identifying the keys for available data
for key, values in {
    0: ['subject status: healthy control'],
    1: ['subject id: HC-1', 'subject id: HC-2', 'subject id: HC-3', 'subject id: HC-4', 'subject id: HC-5', 'subject id: HC-6', 'subject id: pSS-1', 'subject id: pSS-2', 'subject id: pSS-3', 'subject id: pSS-4', 'subject id: pSS-5', 'subject id: pSS-6'],
    2: ['cell type: peripheral blood B cell'],
    3: ['cell subtype: Bm1', 'cell subtype: Naive B cell', 'cell subtype: Pre-GC B cell', 'cell subtype: Memory B cell']
}.items():
    if "subject status" in values[0]:
        trait_row = key

# Define conversion functions
def convert_trait(value):
    if "healthy control" in value:
        return 0
    if "pSS" in value:
        return 1
    return None

# Age and Gender data not found
convert_age = convert_gender = None  # Not available, functions kept for completeness.

# Save cohort information
save_cohort_info('GSE135809', './preprocessed/Sjögrens_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and save clinical feature extraction if available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Sjögrens_Syndrome', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Sjögrens_Syndrome/trait_data/GSE135809.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4030545': [0], 'GSM4030546': [0], 'GSM4030547': [0], 'GSM4030548': [0], 'GSM4030549': [0], 'GSM4030550': [0], 'GSM4030551': [0], 'GSM4030552': [0], 'GSM4030553': [0], 'GSM4030554': [0], 'GSM4030555': [0], 'GSM4030556': [0], 'GSM4030557': [0], 'GSM4030558': [0], 'GSM4030559': [0], 'GSM4030560': [0], 'GSM4030561': [0], 'GSM4030562': [0], 'GSM4030563': [0], 'GSM4030564': [0], 'GSM4030565': [0], 'GSM4030566': [0], 'GSM4030567': [0], 'GSM4030568': [0], 'GSM4030569': [0], 'GSM4030570': [0], 'GSM4030571': [0], 'GSM4030572': [0], 'GSM4030573': [0], 'GSM4030574': [0], 'GSM4030575': [0], 'GSM4030576': [0], 'GSM4030577': [0], 'GSM4030578': [0], 'GSM4030579': [0], 'GSM4030580': [0], 'GSM4030581': [0], 'GSM4030582': [0], 'GSM4030583': [0], 'GSM4030584': [0], 'GSM4030585': [0], 'GSM4030586': [0], 'GSM4030587': [0], 'GSM4030588': [0], 'GSM4030589': [0], 'GSM4030590': [0], 'GSM4030591': [0], 'GSM4030592': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifying the appropriate keys for gene mapping
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression data with gene symbols
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Sjögrens_Syndrome/gene_data/GSE135809.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Sjögrens_Syndrome')

# 4. Save the cohort information.
json_path = './preprocessed/Sjögrens_Syndrome/cohort_info.json'
save_cohort_info('GSE135809', json_path, True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Sjögrens_Syndrome/GSE135809.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Sjögrens_Syndrome':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Sjögrens_Syndrome' in this dataset is severely biased.

